<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Neural Networks

## *Data Science Unit 4 Sprint 2 Assignment 1*

## Define the Following:
You can add image, diagrams, whatever you need to ensure that you understand the concepts below.

### Input Layer: Where the data from our dataset gets put into the neural network.
### Hidden Layer: A layer where data is put in and data comes out. We can't inspect what is happening because it is an invisible layer. Helps with learning more complex relationships.
### Output Layer: Where you get your answer. 
### Neuron: A neuron receives input and passes their signal to the next layer.
### Weight: It's similar to slope in a linear function. Each input node has a weight attached.
### Activation Function: Decides whether a cell "fires" or not. They decide how much signal to pass onto the next layer.
### Node Map: A diagram that shows the path that the neural net follows. 
### Perceptron: It is a single-node neural network. It takes in n inputs and spits out an output. In this neural network, it takes the sum of the inputs multiplied by the weights and summed with a bias value.


## Inputs -> Outputs

### Explain the flow of information through a neural network from inputs to outputs. Be sure to include: inputs, weights, bias, and activation functions. How does it all flow from beginning to end?

#### Start with the inputs. Each input is multiplied by a weight and they are summed together. Take the sum and push it through an activation function. Whatever comes out is summed with a bias value and then you get your final output.

## Write your own perceptron code that can correctly classify (99.0% accuracy) a NAND gate. 

| x1 | x2 | y |
|----|----|---|
| 0  | 0  | 1 |
| 1  | 0  | 1 |
| 0  | 1  | 1 |
| 1  | 1  | 0 |

In [10]:
##### Your Code Here #####
import numpy as np

np.random.seed(66)

inputs = np.array([
    [0, 0, 1],
    [1, 0, 1],
    [0, 1, 1],
    [1, 1, 1]
])

correct_outputs = [[1], [1], [1], [0]]

In [57]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    sx = sigmoid(x)
    return sx * (1-sx)

In [136]:
weights = 2*np.random.random((3,1)) - 1
weights

array([[-0.69142484],
       [-0.73260088],
       [-0.27462907]])

In [137]:
for iter in range(10000):
    
    weighted_sum = np.dot(inputs, weights)
    
    activated_output = sigmoid(weighted_sum)
    
    error = correct_outputs - activated_output
    
    adjustments = error * sigmoid_derivative(activated_output)
    
    weights += np.dot(inputs.T, adjustments)
#     print(weights)
    
print("Weights after training")
print(weights)

print("Output after training")
print(activated_output)

Weights after training
[[-11.8403001 ]
 [-11.8403001 ]
 [ 17.80931455]]
Output after training
[[0.99999998]
 [0.99744951]
 [0.99744951]
 [0.0028116 ]]


## Implement your own Perceptron Class and use it to classify a binary dataset: 
- [The Pima Indians Diabetes dataset](https://raw.githubusercontent.com/ryanleeallred/datasets/master/diabetes.csv) 

You may need to search for other's implementations in order to get inspiration for your own. There are *lots* of perceptron implementations on the internet with varying levels of sophistication and complexity. Whatever your approach, make sure you understand **every** line of your implementation and what its purpose is.

In [391]:
diabetes = pd.read_csv('https://raw.githubusercontent.com/ryanleeallred/datasets/master/diabetes.csv')
diabetes.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


Although neural networks can handle non-normalized data, scaling or normalizing your data will improve your neural network's learning speed. Try to apply the sklearn `MinMaxScaler` or `Normalizer` to your diabetes dataset. 

In [392]:
from sklearn.preprocessing import MinMaxScaler, Normalizer

X = diabetes.drop(columns='Outcome')
y = diabetes['Outcome'].values

In [393]:
X.shape

(768, 8)

In [394]:
X.insert(8, 'b', np.ones((768, 1)))

In [395]:
X = X.values

In [396]:
weights = 2*np.random.random((9,1)) - 1
weights

array([[ 0.04406061],
       [ 0.6355258 ],
       [ 0.8521554 ],
       [-0.97667362],
       [-0.02100396],
       [ 0.52812171],
       [ 0.45410062],
       [ 0.54056725],
       [-0.46619218]])

In [398]:
for iter in range(10):
    
    weighted_sum = np.dot(X, weights)
    
    activated_output = sigmoid(weighted_sum)
    
    error = y - activated_output
    
    adjustments = error * sigmoid_derivative(activated_output)
    
    weights = weights + np.dot(X.T, adjustments)
    
print("Weights after training")
print(weights)

print("Output after training")
print(activated_output)

Weights after training
[[ 4.40606076e-02 -5.80550978e+02  4.40606076e-02 ... -5.80550978e+02
   4.40606076e-02 -5.80550978e+02]
 [ 6.35525797e-01 -1.82541926e+04  6.35525797e-01 ... -1.82541926e+04
   6.35525797e-01 -1.82541926e+04]
 [ 8.52155404e-01 -1.04339330e+04  8.52155404e-01 ... -1.04339330e+04
   8.52155404e-01 -1.04339330e+04]
 ...
 [ 4.54100624e-01 -7.07982606e+01  4.54100624e-01 ... -7.07982606e+01
   4.54100624e-01 -7.07982606e+01]
 [ 5.40567245e-01 -5.01876548e+03  5.40567245e-01 ... -5.01876548e+03
   5.40567245e-01 -5.01876548e+03]
 [-4.66192177e-01 -1.51464157e+02 -4.66192177e-01 ... -1.51464157e+02
  -4.66192177e-01 -1.51464157e+02]]
Output after training
[[1. 0. 1. ... 0. 1. 0.]
 [1. 0. 1. ... 0. 1. 0.]
 [1. 0. 1. ... 0. 1. 0.]
 ...
 [1. 0. 1. ... 0. 1. 0.]
 [1. 0. 1. ... 0. 1. 0.]
 [1. 0. 1. ... 0. 1. 0.]]


## Stretch Goals:

- Research "backpropagation" to learn how weights get updated in neural networks (tomorrow's lecture). 
- Implement a multi-layer perceptron. (for non-linearly separable classes)
- Try and implement your own backpropagation algorithm.
- What are the pros and cons of the different activation functions? How should you decide between them for the different layers of a neural network?